<a href="https://colab.research.google.com/github/cwkruppo/Coursera_Capstone/blob/master/CK_Battle_of_the_Neighbourhoods_Vaccine_Sites_Near_Public_Transit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Final Notebook Submission for Foursquare API for IBM Capstone Project
## The goal of the project is to display pharmacies within 3/4mi of public train stations to facilitate coronavirus vaccination efforts.
## Please see the full report and presentation for complete details.

## What follows is the import of train station data from Wikipedia API and merged with the NJ Transit API dataset in Morris County, New Jersey.
## Then, a series of API calls for the pharmacies within range of each station plotted on a Folium map, using venue data from Foursquare's API.

In [2]:
## Import train station stop name list from Wikipedia API, clean and format in for lat/lon
#!/usr/bin/python3

"""
    get_subcategories.py

    MediaWiki API Demos
    Demo of `Categorymembers` module : Get ten subcategories of a category

    MIT License
"""

import requests

S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    "action": "query",
    "cmtitle": "Category:Railway_stations_in_Morris_County,_New_Jersey",
    "cmtype": "page",
    "list": "categorymembers",
    "cmlimit": "max",
    "format": "json"
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

PAGES = DATA["query"]["categorymembers"]

for page in PAGES:
    
    wikilist = PAGES
    from pandas import DataFrame

dfwiki = wikilist

dfwiki = DataFrame (dfwiki,columns=['pageid','ns','title'])
dfwiki.drop(['pageid', 'ns'], axis=1, inplace=True)
dfwiki.rename(columns={'title': 'stop_name'}, inplace=True)
dfwiki = dfwiki[~dfwiki['stop_name'].str.contains('Railroad')]
dfwiki['stop_name'] = dfwiki['stop_name'].str.upper()
dfwiki = dfwiki.stop_name.str.replace(' STATION.*', '')
## import station data from NJ Transit
import pandas as pd
dfstops = pd.read_table("https://raw.githubusercontent.com/cwkruppo/Coursera_Capstone/master/stops.txt", sep=',', header=0)
dfstops.drop(['stop_id', 'stop_desc', 'zone_id'], axis=1, inplace=True)
dfstops = dfstops[~dfstops['stop_name'].str.contains('LIGHT RAIL')]
## Merge list of stations from NJ Transit with list of county stations from Wikipedia
mergedstops = pd.merge(left=dfstops, right=dfwiki, left_on='stop_name', right_on='stop_name')
mergedstops

,stop_code,stop_name,stop_lat,stop_lon
0,95020,BOONTON,40.903378,-74.407736
1,95030,CONVENT,40.779038,-74.443435
2,95035,DOVER,40.883415,-74.555887
3,95067,LAKE HOPATCONG,40.904219,-74.665697
4,95077,MADISON,40.757028,-74.415105
5,95088,MILLINGTON,40.673513,-74.523606
6,95091,MORRIS PLAINS,40.828637,-74.478197
7,95092,MORRISTOWN,40.797113,-74.474086
8,95094,MOUNT TABOR,40.875904,-74.481915
9,95147,TOWACO,40.922809,-74.343842


In [3]:
## Import some of the key packages for the project

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 81kB 3.8MB/s 
  Created wheel for folium: filename=folium-0.5.0-cp37-none-any.whl size=76240 sha256=4186e0e3129921f514bcef3c1bab3edf76c85d5b31fc64095bebae9a3fcf7578
  Stored in directory: /root/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.5.0 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
Folium installed
Libraries imported.


In [18]:
# @hidden
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('.cm-comment:contains(@hidden)').closest('div.input').hide();
 } else {
 $('.cm-comment:contains(@hidden)').closest('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')
#@title
## Assign Foursquare login credentials
# @hidden_cell

CLIENT_ID = 'BCLLIS3FHZXMBORMFLCQMYFH1YDVZVO0F2BMJ4KONFEAZ021' # your Foursquare ID
CLIENT_SECRET = 'ZYUFKQJASGY1DKGUDNMJJH0FJFSV11NPAS22IFXIHS5OPD5F' # your Foursquare Secret
##code SQV35FLC2CIZWHSASCFI4MXNKDJC1MAT5DX3IDLJJMMT3BRN
ACCESS_TOKEN = 'WDVVTN0TFM1FIZRC2BJOLJYYVQJ2PJVWJ22HFU1BOFCHJ4HZ' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Foursquare API credentials submitted via hidden cell')

Foursquare API credentials submitted via hidden cell


In [8]:
## Iterate the train station dataframe column rows and assign each to a dictonary - station name, latitute,
## Station Names
stationsdict = list(iter(mergedstops.stop_name))
dict= {}
for i in stationsdict:
  dict[i] = i
## Station Lat
stationlatdict = list(iter(mergedstops.stop_lat))
dict= {}
for i in stationlatdict:
  dict[i] = i
## Station Lon
stationlondict = list(iter(mergedstops.stop_lon))
dict= {}
for i in stationlondict:
  dict[i] = i
##values = dict.values()
##values

In [9]:
## Convert dictionary values to individual values. This is a process that can be automated in the future.

station1 = stationsdict[0]
stationlat1 = stationlatdict[0]
stationlon1 = stationlondict[0]

station2 = stationsdict[1]
stationlat2 = stationlatdict[1]
stationlon2 = stationlondict[1]

station3 = stationsdict[2]
stationlat3 = stationlatdict[2]
stationlon3 = stationlondict[2]

station4 = stationsdict[3]
stationlat4 = stationlatdict[3]
stationlon4 = stationlondict[3]

station5 = stationsdict[4]
stationlat5 = stationlatdict[4]
stationlon5 = stationlondict[4]

station6 = stationsdict[5]
stationlat6 = stationlatdict[5]
stationlon6 = stationlondict[5]

station7 = stationsdict[6]
stationlat7 = stationlatdict[6]
stationlon7 = stationlondict[6]

station8 = stationsdict[7]
stationlat8 = stationlatdict[7]
stationlon8 = stationlondict[7]

station9 = stationsdict[8]
stationlat9 = stationlatdict[8]
stationlon9 = stationlondict[8]

station10 = stationsdict[9]
stationlat10 = stationlatdict[9]
stationlon10 = stationlondict[9]

station11 = stationsdict[10]
stationlat11 = stationlatdict[10]
stationlon11 = stationlondict[10]

print("Station Data Assigned")

Station Data Assigned


In [12]:
## Generate the Folium map, center it on Morris County, NJ; generate circles for each station in red; make Foursquare API calls and plot the results as circle markers in blue.

## Map Center - name origin location
address = 'Morris County, NJ'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitudex = location.latitude
longitudex = location.longitude

## Foursquare Query 1 - name origin location
##address = 'Morris County, NJ'
##geolocator = Nominatim(user_agent="foursquare_agent")
##location = geolocator.geocode(address)
latitude = stationlat1
longitude = stationlon1

## name search parameters
search_query = 'Pharmacy'
radius = 1200

## define corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

## Convert to pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

## Filter results to only areas of interest
# keep only columns that include venue name, and anything that is associated with location
try:
  filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
except KeyError as ke:
   pass
try:
  dataframe_filtered = dataframe.loc[:, filtered_columns]
except KeyError as ke:
   pass

# function that extracts the category of the venue
def get_category_type(row):
  try:
        categories_list = row['categories']
  except:
        categories_list = row['venue.categories']
        
  if len(categories_list) == 0:
        return None
  else:
        return categories_list[0]['name']

# filter the category for each row
try:
  dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
except KeyError as ke:
  pass
except TypeError as te:
   pass

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

## Visualize the data
dataframe_filtered.name

venues_map = folium.Map(location=[latitudex, longitudex], tiles="Stamen Toner", zoom_start=10) # generate map centred on Morris County, NJ

# add a red circle marker to represent the train station 1
folium.Circle(
    [stationlat1, stationlon1],
    radius=1200,
    color='red',
    popup= station1,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the train station 2
folium.Circle(
    [stationlat2, stationlon2],
    radius=1200,
    color='red',
    popup= station2,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the train station 3
folium.Circle(
    [stationlat3, stationlon3],
    radius=1200,
    color='red',
    popup= station3,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the train station 4
folium.Circle(
    [stationlat4, stationlon4],
    radius=1200,
    color='red',
    popup= station4,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the train station 5
folium.Circle(
    [stationlat5, stationlon5],
    radius=1200,
    color='red',
    popup= station5,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the train station 6
folium.Circle(
    [stationlat6, stationlon6],
    radius=1200,
    color='red',
    popup= station6,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the train station 7
folium.Circle(
    [stationlat7, stationlon7],
    radius=1200,
    color='red',
    popup= station7,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the train station 8
folium.Circle(
    [stationlat8, stationlon8],
    radius=1200,
    color='red',
    popup= station8,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the train station 9
folium.Circle(
    [stationlat9, stationlon9],
    radius=1200,
    color='red',
    popup= station9,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the train station 10
folium.Circle(
    [stationlat10, stationlon10],
    radius=1200,
    color='red',
    popup= station10,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the train station 11
folium.Circle(
    [stationlat11, stationlon11],
    radius=1200,
    color='red',
    popup= station11,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the query 1 pharmacies as blue circle markers
for name, lat, lng, label in zip(dataframe_filtered.name, dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name+" - "+dataframe_filtered.city+", "+dataframe_filtered.state):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    ##
    ##
    ##
    
## Foursquare Query 2 - name origin location
##address = 'Morris County, NJ'
##geolocator = Nominatim(user_agent="foursquare_agent")
##location = geolocator.geocode(address)
latitude = stationlat2
longitude = stationlon2

## name search parameters
search_query = 'Pharmacy'
radius = 1200

## define corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

## Convert to pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues
# tranform venues into a dataframe
dataframe = json_normalize(venues)

## Filter results to only areas of interest
# keep only columns that include venue name, and anything that is associated with location
try:
  filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
except KeyError as ke:
   pass
try:
  dataframe_filtered = dataframe.loc[:, filtered_columns]
except KeyError as ke:
   pass

# function that extracts the category of the venue
def get_category_type(row):
  try:
        categories_list = row['categories']
  except:
        categories_list = row['venue.categories']
        
  if len(categories_list) == 0:
        return None
  else:
        return categories_list[0]['name']

# filter the category for each row
try:
  dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
except KeyError as ke:
  pass
except TypeError as te:
   pass

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

## Visualize the data
dataframe_filtered.name

    # add the query 2 pharmacies as blue circle markers
for name, lat, lng, label in zip(dataframe_filtered.name, dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name+" - "+dataframe_filtered.city+", "+dataframe_filtered.state):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    ##
    ##
    ##


## Foursquare Query 3 - name origin location
##address = 'Morris County, NJ'
##geolocator = Nominatim(user_agent="foursquare_agent")
##location = geolocator.geocode(address)
latitude = stationlat3
longitude = stationlon3

## name search parameters
search_query = 'Pharmacy'
radius = 1200

## define corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

## Convert to pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues
# tranform venues into a dataframe
dataframe = json_normalize(venues)

## Filter results to only areas of interest
# keep only columns that include venue name, and anything that is associated with location
try:
  filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
except KeyError as ke:
   pass
try:
  dataframe_filtered = dataframe.loc[:, filtered_columns]
except KeyError as ke:
   pass

# function that extracts the category of the venue
def get_category_type(row):
  try:
        categories_list = row['categories']
  except:
        categories_list = row['venue.categories']
        
  if len(categories_list) == 0:
        return None
  else:
        return categories_list[0]['name']

# filter the category for each row
try:
  dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
except KeyError as ke:
  pass
except TypeError as te:
   pass

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

## Visualize the data
dataframe_filtered.name

# add the query 3 pharmacies as blue circle markers
for name, lat, lng, label in zip(dataframe_filtered.name, dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name+" - "+dataframe_filtered.city+", "+dataframe_filtered.state):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    ##
    ##
    ##

## Foursquare Query 4 - name origin location
##address = 'Morris County, NJ'
##geolocator = Nominatim(user_agent="foursquare_agent")
##location = geolocator.geocode(address)
latitude = stationlat4
longitude = stationlon4

## name search parameters
search_query = 'Pharmacy'
radius = 1200

## define corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

## Convert to pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues
# tranform venues into a dataframe
dataframe = json_normalize(venues)

## Filter results to only areas of interest
# keep only columns that include venue name, and anything that is associated with location
try:
  filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
except KeyError as ke:
   pass
try:
  dataframe_filtered = dataframe.loc[:, filtered_columns]
except KeyError as ke:
   pass

# function that extracts the category of the venue
def get_category_type(row):
  try:
        categories_list = row['categories']
  except:
        categories_list = row['venue.categories']
        
  if len(categories_list) == 0:
        return None
  else:
        return categories_list[0]['name']

# filter the category for each row
try:
  dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
except KeyError as ke:
  pass
except TypeError as te:
   pass

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

## Visualize the data
dataframe_filtered.name

# add the query 4 pharmacies as blue circle markers
for name, lat, lng, label in zip(dataframe_filtered.name, dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name+" - "+dataframe_filtered.city+", "+dataframe_filtered.state):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    ##
    ##
    ##

## Foursquare Query 5 - name origin location
##address = 'Morris County, NJ'
##geolocator = Nominatim(user_agent="foursquare_agent")
##location = geolocator.geocode(address)
latitude = stationlat5
longitude = stationlon5

## name search parameters
search_query = 'Pharmacy'
radius = 1200

## define corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

## Convert to pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues
# tranform venues into a dataframe
dataframe = json_normalize(venues)

## Filter results to only areas of interest
# keep only columns that include venue name, and anything that is associated with location
try:
  filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
except KeyError as ke:
   pass
try:
  dataframe_filtered = dataframe.loc[:, filtered_columns]
except KeyError as ke:
   pass

# function that extracts the category of the venue
def get_category_type(row):
  try:
        categories_list = row['categories']
  except:
        categories_list = row['venue.categories']
        
  if len(categories_list) == 0:
        return None
  else:
        return categories_list[0]['name']

# filter the category for each row
try:
  dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
except KeyError as ke:
  pass
except TypeError as te:
   pass

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

## Visualize the data
dataframe_filtered.name

# add the query 5 pharmacies as blue circle markers
for name, lat, lng, label in zip(dataframe_filtered.name, dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name+" - "+dataframe_filtered.city+", "+dataframe_filtered.state):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    ##
    ##
    ##

## Foursquare Query 6 - name origin location
##address = 'Morris County, NJ'
##geolocator = Nominatim(user_agent="foursquare_agent")
##location = geolocator.geocode(address)
latitude = stationlat6
longitude = stationlon6

## name search parameters
search_query = 'Pharmacy'
radius = 1200

## define corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

## Convert to pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues
# tranform venues into a dataframe
dataframe = json_normalize(venues)

## Filter results to only areas of interest
# keep only columns that include venue name, and anything that is associated with location
try:
  filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
except KeyError as ke:
   pass
try:
  dataframe_filtered = dataframe.loc[:, filtered_columns]
except KeyError as ke:
   pass

# function that extracts the category of the venue
def get_category_type(row):
  try:
        categories_list = row['categories']
  except:
        categories_list = row['venue.categories']
        
  if len(categories_list) == 0:
        return None
  else:
        return categories_list[0]['name']

# filter the category for each row
try:
  dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
except KeyError as ke:
  pass
except TypeError as te:
   pass

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

## Visualize the data
dataframe_filtered.name

# add the query 6 pharmacies as blue circle markers
for name, lat, lng, label in zip(dataframe_filtered.name, dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name+" - "+dataframe_filtered.city+", "+dataframe_filtered.state):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    ##
    ##
    ##

## Foursquare Query 7 - name origin location
##address = 'Morris County, NJ'
##geolocator = Nominatim(user_agent="foursquare_agent")
##location = geolocator.geocode(address)
latitude = stationlat7
longitude = stationlon7

## name search parameters
search_query = 'Pharmacy'
radius = 1200

## define corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

## Convert to pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues
# tranform venues into a dataframe
dataframe = json_normalize(venues)

## Filter results to only areas of interest
# keep only columns that include venue name, and anything that is associated with location
try:
  filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
except KeyError as ke:
   pass
try:
  dataframe_filtered = dataframe.loc[:, filtered_columns]
except KeyError as ke:
   pass

# function that extracts the category of the venue
def get_category_type(row):
  try:
        categories_list = row['categories']
  except:
        categories_list = row['venue.categories']
        
  if len(categories_list) == 0:
        return None
  else:
        return categories_list[0]['name']

# filter the category for each row
try:
  dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
except KeyError as ke:
  pass
except TypeError as te:
   pass

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

## Visualize the data
dataframe_filtered.name

    # add the query 7 pharmacies as blue circle markers
for name, lat, lng, label in zip(dataframe_filtered.name, dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name+" - "+dataframe_filtered.city+", "+dataframe_filtered.state):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    ##
    ##
    ##

## Foursquare Query 8 - name origin location
##address = 'Morris County, NJ'
##geolocator = Nominatim(user_agent="foursquare_agent")
##location = geolocator.geocode(address)
latitude = stationlat8
longitude = stationlon8

## name search parameters
search_query = 'Walgreens'
radius = 1200

## define corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

## Convert to pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues
# tranform venues into a dataframe
dataframe = json_normalize(venues)

## Filter results to only areas of interest
# keep only columns that include venue name, and anything that is associated with location
try:
  filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
except KeyError as ke:
   pass
try:
  dataframe_filtered = dataframe.loc[:, filtered_columns]
except KeyError as ke:
   pass

# function that extracts the category of the venue
def get_category_type(row):
  try:
        categories_list = row['categories']
  except:
        categories_list = row['venue.categories']
        
  if len(categories_list) == 0:
        return None
  else:
        return categories_list[0]['name']

# filter the category for each row
try:
  dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
except KeyError as ke:
  pass
except TypeError as te:
   pass

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

## Visualize the data
dataframe_filtered.name

    # add the query 8 pharmacies as blue circle markers
for name, lat, lng, label in zip(dataframe_filtered.name, dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name+" - "+dataframe_filtered.city+", "+dataframe_filtered.state):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    ##
    ##
    ##

## Foursquare Query 9 - name origin location
##address = 'Morris County, NJ'
##geolocator = Nominatim(user_agent="foursquare_agent")
##location = geolocator.geocode(address)
latitude = stationlat9
longitude = stationlon9

## name search parameters
search_query = 'CVS'
radius = 1200

## define corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

## Convert to pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues
# tranform venues into a dataframe
dataframe = json_normalize(venues)

## Filter results to only areas of interest
# keep only columns that include venue name, and anything that is associated with location
try:
  filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
except KeyError as ke:
   pass
try:
  dataframe_filtered = dataframe.loc[:, filtered_columns]
except KeyError as ke:
   pass

# function that extracts the category of the venue
def get_category_type(row):
  try:
        categories_list = row['categories']
  except:
        categories_list = row['venue.categories']
        
  if len(categories_list) == 0:
        return None
  else:
        return categories_list[0]['name']

# filter the category for each row
try:
  dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
except KeyError as ke:
  pass
except TypeError as te:
   pass

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

## Visualize the data
dataframe_filtered.name

    # add the query 9 pharmacies as blue circle markers
for name, lat, lng, label in zip(dataframe_filtered.name, dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name+" - "+dataframe_filtered.city+", "+dataframe_filtered.state):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    ##
    ##
    ##

## Foursquare Query 10 - name origin location
##address = 'Morris County, NJ'
##geolocator = Nominatim(user_agent="foursquare_agent")
##location = geolocator.geocode(address)
latitude = stationlat10
longitude = stationlon10

## name search parameters
search_query = 'Pharmacy'
radius = 1200

## define corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

## Convert to pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues
# tranform venues into a dataframe
dataframe = json_normalize(venues)

## Filter results to only areas of interest
# keep only columns that include venue name, and anything that is associated with location
try:
  filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
except KeyError as ke:
   pass
try:
  dataframe_filtered = dataframe.loc[:, filtered_columns]
except KeyError as ke:
   pass

# function that extracts the category of the venue
def get_category_type(row):
  try:
        categories_list = row['categories']
  except:
        categories_list = row['venue.categories']
        
  if len(categories_list) == 0:
        return None
  else:
        return categories_list[0]['name']

# filter the category for each row
try:
  dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
except KeyError as ke:
  pass
except TypeError as te:
   pass

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

## Visualize the data
dataframe_filtered.name

    # add the query 10 pharmacies as blue circle markers
for name, lat, lng, label in zip(dataframe_filtered.name, dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name+" - "+dataframe_filtered.city+", "+dataframe_filtered.state):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    ##
    ##
    ##

## Foursquare Query 11 - name origin location
##address = 'Morris County, NJ'
##geolocator = Nominatim(user_agent="foursquare_agent")
##location = geolocator.geocode(address)
latitude = stationlat11
longitude = stationlon11

## name search parameters
search_query = 'Pharmacy'
radius = 1200

## define corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

## Convert to pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues
# tranform venues into a dataframe
dataframe = json_normalize(venues)

## Filter results to only areas of interest
# keep only columns that include venue name, and anything that is associated with location
try:
  filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
except KeyError as ke:
   pass
try:
  dataframe_filtered = dataframe.loc[:, filtered_columns]
except KeyError as ke:
   pass

# function that extracts the category of the venue
def get_category_type(row):
  try:
        categories_list = row['categories']
  except:
        categories_list = row['venue.categories']
        
  if len(categories_list) == 0:
        return None
  else:
        return categories_list[0]['name']

# filter the category for each row
try:
  dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
except KeyError as ke:
  pass
except TypeError as te:
   pass

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

## Visualize the data
dataframe_filtered.name

    # add the query 11 pharmacies as blue circle markers
for name, lat, lng, label in zip(dataframe_filtered.name, dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name+" - "+dataframe_filtered.city+", "+dataframe_filtered.state):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    ###
    ###
    ###
 
# display map
print('Location loaded, search parameters defined, url generated, results saved & converted to dataframe, map generated!')
venues_map

## No Nearby Stations - passed 2, 4, 6, 10
## Blank map results when queried - 8 & 9
## Contrained search to pass - 8 (Walgreens), 9 (CVS)
##
## For some venue results, the html_id for the specific venue location throws a syntax error 
## because the result from the API is missing a ")" and therefore cannot be displayed and stops the map from generating.
## This is even though the result is in the dataframe. So limiting the results to specific pharmacy names for that search
## allows the map to generate properly.
## -CK


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:228: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:304: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:379: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:454: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:529: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ip

Location loaded, search parameters defined, url generated, results saved & converted to dataframe, map generated!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:904: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [11]:
###